In [1]:
!export CUDA_VISIBLE_DEVICES=0

In [1]:
import os
os.environ['CUDA_VISIBLE_DEVICES']='0'
# os.environ['CUDA_VISIBLE_DEVICES']='0,1'
os.environ['CUDA_LAUNCH_BLOCKING']='1'
import torch
# torch.cuda.set_device(0)
print(torch.cuda.current_device())


0


In [2]:
torch.cuda.is_available()

True

In [3]:
torch.cuda.device_count()

1

In [4]:
torch.cuda.current_device()

0

In [5]:
# cudaSetDevice(0)
#dataset transcript要在dataset資好夾調
#cuda 在basesolver init調
#dataset要在data.py設定

In [1]:
import os
os.environ['CUDA_VISIBLE_DEVICES']='1'
# os.environ['CUDA_VISIBLE_DEVICES']='0,1'
os.environ['CUDA_LAUNCH_BLOCKING']='1'
import torch
# torch.cuda.set_device(0)
print(torch.cuda.current_device())
#! python
# -*- coding: utf-8 -*-
# Author: kun
# @Time: 2019-10-29 20:29

import yaml
import torch
import argparse
import numpy as np
import pandas as pd

dfs = []
alldf =pd.DataFrame()

class Para(object):
    a=1

def force_cudnn_initialization():
    s = 32
    dev = torch.device('cuda:0')
    torch.nn.functional.conv2d(torch.zeros(s, s, s, s, device=dev), torch.zeros(s, s, s, s, device=dev))
    
#force_cudnn_initialization()
def main():
    # For reproducibility, comment these may speed up training
    torch.backends.cudnn.deterministic = True
    torch.backends.cudnn.benchmark = False

    # Arguments
#     parser = argparse.ArgumentParser(description='Training E2E asr.')
#     parser.add_argument('--config', type=str, help='Path to experiment config.')
#     parser.add_argument('--name', default=None, type=str, help='Name for logging.')
#     parser.add_argument('--logdir', default='log/', type=str,
#                         help='Logging path.', required=False)
#     parser.add_argument('--ckpdir', default='ckpt/', type=str,
#                         help='Checkpoint path.', required=False)
#     parser.add_argument('--outdir', default='result/', type=str,
#                         help='Decode output path.', required=False)
#     parser.add_argument('--load', default=None, type=str,
#                         help='Load pre-trained model (for training only)', required=False)
#     parser.add_argument('--seed', default=0, type=int,
#                         help='Random seed for reproducable results.', required=False)
#     parser.add_argument('--cudnn-ctc', action='store_true',
#                         help='Switches CTC backend from torch to cudnn')
#     parser.add_argument('--njobs', default=32, type=int,
#                         help='Number of threads for dataloader/decoding.', required=False)
#     parser.add_argument('--cpu', action='store_true', help='Disable GPU training.')
#     parser.add_argument('--no-pin', action='store_true',
#                         help='Disable pin-memory for dataloader')
#     parser.add_argument('--test', action='store_true', help='Test the model.')
#     parser.add_argument('--no-msg', action='store_true', help='Hide all messages.')
#     parser.add_argument('--lm', action='store_true',
#                         help='Option for training RNNLM.')
#     # Following features in development.
#     parser.add_argument('--amp', action='store_true', help='Option to enable AMP.')
#     parser.add_argument('--reserve-gpu', default=0, type=float,
#                         help='Option to reserve GPU ram for training.')
#     parser.add_argument('--jit', action='store_true',
#                         help='Option for enabling jit in pytorch. (feature in development)')
#     ###
#     paras = parser.parse_args()
    paras = Para()
#     paras.config = './config/aishell_asr_example_lstm4atthead1-test.yaml'
#     paras.name = None
#     paras.logdir = 'log/'
#     paras.ckpdir = 'ckpt/'
#     paras.outdir = 'result/'
#     paras.load = None
#     paras.seed = 0
#     paras.cudnn_ctc = False
#     paras.cpu = False
#     paras.no_pin = False
#     paras.test = True
#     paras.no_msg = False
#     paras.lm = False
#     paras.amp = False
#     paras.reserve_gpu = 0
#     paras.jit = False
    setattr(paras, 'config', './config/cv11Lu_asr_lstm4atthead_allvocab-biclass-test-5-1.yaml')
    setattr(paras, 'name', None)
    setattr(paras, 'logdir', 'log/')
    setattr(paras, 'ckpdir', 'ckpt/')
    setattr(paras, 'outdir', 'result/')
    setattr(paras, 'load', None)
    setattr(paras, 'seed', 0)
    setattr(paras, 'cudnn_ctc', False)
    setattr(paras, 'njobs',8)
    setattr(paras, 'cpu', False)
    setattr(paras, 'no_pin', False)
    setattr(paras, 'test', True)
    setattr(paras, 'no_msg', False)
    setattr(paras, 'lm', False)
    setattr(paras, 'amp', False)
    setattr(paras, 'reserve_gpu', 8)
    setattr(paras, 'jit', False)
    setattr(paras, 'gpu', not paras.cpu)
    setattr(paras, 'pin_memory', not paras.no_pin)
    setattr(paras, 'verbose', not paras.no_msg)
    setattr(paras, 'finetune', False)
    setattr(paras, 'binaryClass', True)
    force_cudnn_initialization()

    config = yaml.load(open(paras.config, 'r'), Loader=yaml.FullLoader)

    np.random.seed(paras.seed)
    torch.manual_seed(paras.seed)
    if torch.cuda.is_available():
        torch.cuda.manual_seed_all(paras.seed)

    # Hack to preserve GPU ram just incase OOM later on server
    if paras.gpu and paras.reserve_gpu > 0:
        buff = torch.randn(int(paras.reserve_gpu * 1e9 // 4)).cuda()
        del buff

    if paras.lm:
        # Train RNNLM
        from train_lm import Solver

        mode = 'train'
    else:
        if paras.test:
            # Test ASR
            assert paras.load is None, 'Load option is mutually exclusive to --test'
            from test_binaryclass import Solver

            mode = 'test'
        elif paras.finetune:
            assert paras.load is not None
            from finetune_asr import Solver
            mode = 'train'
        elif paras.binaryClass:
            assert paras.load is not None
            from train_binaryclass import Solver
            mode = 'train'
        else:
            # Train ASR
            from train_asr import Solver

            mode = 'train'

    print("\nUsing {} mode\n".format(mode))
    for idx in range(4):
        paras.config = f'./config/cv11Lu_asr_lstm4atthead_allvocab-biclass-test-4-{idx+1}.yaml'
        config = yaml.load(open(paras.config, 'r'), Loader=yaml.FullLoader)
        solver = Solver(config, paras, mode)

        solver.load_data()
    #     solver.print_model()
        solver.set_model()
        names, hyps, txts, ans = solver.exec()
        dfs.append(pd.DataFrame(
            {'name': names,
             'hyps': hyps,
             'truth': txts,
             'ifcorrect': ans
            }))
        print(dfs[idx])
        print(names)
        print(hyps)
        print(txts)
        print(ans)
        


0


In [2]:
main()


Using test mode

[INFO] Evaluating result of tr. config @ ../LAS_Mandarin_PyTorch-master/config/cv11Lu_asr_lstm4atthead_allvocab-biclass2-4fold-1.yaml
Load data for training/validation, store tokenizer and input/output shape
Prepare dataloader for training/validation
Interface for creating all kinds of dataset
import VAGDataset as Dataset
[VAGDataset] path: /home/jupyter-jason5/data_process, split: ['CTT4-1']
Mozillacv11Dataset CTT4-1 found wav data: 21
text len: 21
remove None, then wav data: 21, text len: 21
[VAGDataset] path: /home/jupyter-jason5/data_process, split: ['CTTtrain']
Mozillacv11Dataset CTTtrain found wav data: 73
text len: 73
remove None, then wav data: 73, text len: 73
[INFO] Data spec. | Corpus = vag (from /home/jupyter-jason5/data_process)                                  
[INFO]            | Train sets = ['CTTtrain']	| Number of utts = 73                                        
[INFO]            | Dev sets = ['CTT4-1']	| Number of utts = 21                         

/home/jupyter-jason5/LAS_Mandarin_PyTorch-master/core/module.py:51: UserWarning: __floordiv__ is deprecated, and its behavior will change in a future version of pytorch. It currently rounds toward 0 (like the 'trunc' function NOT 'floor'). This results in incorrect rounding for negative values. To keep the current behavior, use torch.div(a, b, rounding_mode='trunc'), or for actual floor division, use torch.div(a, b, rounding_mode='floor').
  feat_len = feat_len // 4


['78CTT']   0.41415587067604065   0.0   True
['33CTT']   0.42483144998550415   1.0   False
['14CTT']   0.4004267454147339   0.0   True
['85CTT']   0.3983447253704071   0.0   True
['58CTT']   0.3867952525615692   1.0   False
['68CTT']   0.41581910848617554   0.0   True
['86CTT']   0.40345248579978943   0.0   True
['35CTT']   0.39768218994140625   1.0   False
['65CTT']   0.3967791497707367   1.0   False
['93CTT']   0.40124455094337463   0.0   True
['92CTT']   0.4071401357650757   0.0   True
['15CTT']   0.4093697965145111   0.0   True
['82CTT']   0.39788818359375   0.0   True
['18CTT']   0.4136504530906677   1.0   False
['48CTT']   0.4040530025959015   1.0   False
['46CTT']   0.40205609798431396   0.0   True
['77CTT']   0.4007532298564911   0.0   True
['59CTT']   0.38536766171455383   1.0   False
['64CTT']   0.4197952151298523   1.0   False
['5CTT']   0.42104190587997437   1.0   False
['76CTT']   0.4100899398326874   0.0   True
[INFO] All done !                                            

/home/jupyter-jason5/LAS_Mandarin_PyTorch-master/core/module.py:51: UserWarning: __floordiv__ is deprecated, and its behavior will change in a future version of pytorch. It currently rounds toward 0 (like the 'trunc' function NOT 'floor'). This results in incorrect rounding for negative values. To keep the current behavior, use torch.div(a, b, rounding_mode='trunc'), or for actual floor division, use torch.div(a, b, rounding_mode='floor').
  feat_len = feat_len // 4


['13CTT']   0.4156706631183624   0.0   True
['54CTT']   0.41486287117004395   0.0   True
['32CTT']   0.4152521789073944   1.0   False
['79CTT']   0.4145934581756592   0.0   True
['66CTT']   0.4143345057964325   1.0   False
['99CTT']   0.4153295159339905   0.0   True
['10CTT']   0.415571004152298   1.0   False
['96CTT']   0.4173862636089325   0.0   True
['50CTT']   0.41591188311576843   0.0   True
['61CTT']   0.41528189182281494   1.0   False
['72CTT']   0.4155060648918152   0.0   True
['28CTT']   0.4207800030708313   1.0   False
['60CTT']   0.4152500033378601   1.0   False
['27CTT']   0.4158143401145935   1.0   False
['88CTT']   0.4150511920452118   0.0   True
['30CTT']   0.41714727878570557   0.0   True
['37CTT']   0.41427379846572876   1.0   False
['29CTT']   0.4174696207046509   0.0   True
['56CTT']   0.41520971059799194   0.0   True
['26CTT']   0.41957467794418335   1.0   False
[INFO] All done !                                                                                        

/home/jupyter-jason5/LAS_Mandarin_PyTorch-master/core/module.py:51: UserWarning: __floordiv__ is deprecated, and its behavior will change in a future version of pytorch. It currently rounds toward 0 (like the 'trunc' function NOT 'floor'). This results in incorrect rounding for negative values. To keep the current behavior, use torch.div(a, b, rounding_mode='trunc'), or for actual floor division, use torch.div(a, b, rounding_mode='floor').
  feat_len = feat_len // 4


['3CTT']   0.39136236906051636   0.0   True
['53CTT']   0.20494772493839264   0.0   True
['57CTT']   0.3840145170688629   1.0   False
['51CTT']   0.8254131078720093   0.0   False
['12CTT']   0.29806071519851685   1.0   False
['20CTT']   0.23801174759864807   1.0   False
['38CTT']   0.8291063904762268   1.0   True
['34CTT']   0.43355971574783325   0.0   True
['100CTT']   0.9690093398094177   0.0   False
['39CTT']   0.1929672509431839   1.0   False
['70CTT']   0.19260017573833466   0.0   True
['84CTT']   0.19779567420482635   0.0   True
['43CTT']   0.2048722505569458   0.0   True
['89CTT']   0.19107338786125183   0.0   True
['11CTT']   0.9081220030784607   1.0   True
['90CTT']   0.24051494896411896   0.0   True
['98CTT']   0.19494561851024628   0.0   True
['23CTT']   0.8446462154388428   1.0   True
['63CTT']   0.2374391108751297   1.0   False
[INFO] All done !                                                                                          
      name      hyps  truth  ifcorrect


/home/jupyter-jason5/LAS_Mandarin_PyTorch-master/core/module.py:51: UserWarning: __floordiv__ is deprecated, and its behavior will change in a future version of pytorch. It currently rounds toward 0 (like the 'trunc' function NOT 'floor'). This results in incorrect rounding for negative values. To keep the current behavior, use torch.div(a, b, rounding_mode='trunc'), or for actual floor division, use torch.div(a, b, rounding_mode='floor').
  feat_len = feat_len // 4


['21CTT']   3.4658114600460976e-06   1.0   False
['1CTT']   0.00021411078341770917   0.0   True
['74CTT']   2.3426373729762417e-07   0.0   True
['45CTT']   2.3635413981537567e-06   1.0   False
['40CTT']   3.0262153813964687e-05   1.0   False
['22CTT']   0.9999895095825195   1.0   True
['17CTT']   0.9999986886978149   1.0   True
['49CTT']   0.04596006125211716   0.0   True
['81CTT']   0.9999510049819946   0.0   False
['36CTT']   0.9999990463256836   1.0   True
['55CTT']   0.9305168986320496   0.0   False
['24CTT']   0.00020758045138791203   1.0   False
['47CTT']   1.2089240897239506e-07   0.0   True
['6CTT']   1.1343547612341354e-06   0.0   True
['97CTT']   0.9999986886978149   0.0   False
['95CTT']   0.0003275099443271756   0.0   True
['9CTT']   0.9999901056289673   1.0   True
['80CTT']   0.9997285008430481   0.0   False
['87CTT']   3.282496618339792e-05   0.0   True
[INFO] All done !                                                                                          
     name   

In [3]:
for idx, df in enumerate(dfs):
    alldf = pd.concat([alldf, df])
    df.to_csv(f'./biclass_result/4folds5val_nonall-{idx+1}.csv', index=False)
alldf.to_csv(f'./biclass_result/4folds5val_nonall.csv', index=False)    

In [2]:
main()


Using test mode

[INFO] Evaluating result of tr. config @ ../LAS_Mandarin_PyTorch-master/config/cv11Lu_asr_lstm4atthead_allvocab-biclass2-5fold-1.yaml
Load data for training/validation, store tokenizer and input/output shape
Prepare dataloader for training/validation
Interface for creating all kinds of dataset
import VAGDataset as Dataset
[VAGDataset] path: /home/jupyter-jason5/data_process, split: ['CTTtest']
Mozillacv11Dataset CTTtest found wav data: 21
text len: 21
remove None, then wav data: 21, text len: 21
[VAGDataset] path: /home/jupyter-jason5/data_process, split: ['CTTtrain']
Mozillacv11Dataset CTTtrain found wav data: 73
text len: 73
remove None, then wav data: 73, text len: 73
[INFO] Data spec. | Corpus = vag (from /home/jupyter-jason5/data_process)                                  
[INFO]            | Train sets = ['CTTtrain']	| Number of utts = 73                                        
[INFO]            | Dev sets = ['CTTtest']	| Number of utts = 21                      

In [3]:
from sklearn.metrics import classification_report, confusion_matrix
y_true = [0, 0, 1, 0, 1, 0, 0, 1, 0, 1, 1, 0, 0, 1, 0, 0, 1, 0, 0, 1, 0] 
y_pred = [0, 0, 1, 1, 1, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 1, 0, 0, 1, 0]
target_names = ['non-patient', 'patient']
print(classification_report(y_true, y_pred, target_names=target_names))

              precision    recall  f1-score   support

 non-patient       0.92      0.80      0.86        15
     patient       0.62      0.83      0.71         6

    accuracy                           0.81        21
   macro avg       0.77      0.82      0.79        21
weighted avg       0.84      0.81      0.82        21



In [6]:
tn, fp, fn, tp = confusion_matrix(y_true, y_pred).ravel()
print('sensitivuty: ',tp / (fn+tp))
print('specificity: ',tn / (fp+tn))


sensitivuty:  0.8333333333333334
specificity:  0.8


In [10]:
import numpy as np
from sklearn import metrics
y = np.array([0, 0, 1, 0, 1, 0, 0, 1, 0, 1, 1, 0, 0, 1, 0, 0, 1, 0, 0, 1, 0])
pred = np.array([0.0005, 0.1006, 0.8517, 0.9374, 0.9875, 0.2795, 0.0041, 0.0005, 0.0005, 0.2542, 0.0002, 0.0014, 0.0029, 0.9852, 0.0029, 3.6719e-05, 0.9708, 0.0003, 1.2828e-05, 0.9127, 0.0006])
fpr, tpr, thresholds = metrics.roc_curve(y, pred, pos_label=1)
metrics.auc(fpr, tpr)

0.7692307692307693

In [11]:
!python eval.py --file ~/LAS_Mandarin_PyTorch-master/result/mozillacv11_asr_stm4atthead-test_test_output.csv


============  Result of /home/jupyter-jason3/LAS_Mandarin_PyTorch-master/result/mozillacv11_asr_stm4atthead-test_test_output.csv ============
 -----------------------------------------------------------------------
| Statics		|  Truth	|  Prediction	| Abs. Diff.	|
 -----------------------------------------------------------------------
| Avg. # of chars	|  8.01	|  8.10	|  0.16		|
| Avg. # of words	|  1.00	|  1.00	|  0.00		|
 -----------------------------------------------------------------------
 ---------------------------------------------------------------
| Error Rate (%)| Mean		| Std.		| Min./Max.	|
 ---------------------------------------------------------------
| Character	| 7.9479	| 21.68		| 0.00/240.00	|
| Word		| 18.1725	| 38.56		| 0.00/100.00	|
 ---------------------------------------------------------------
Note : If the text unit is phoneme, WER = PER and CER is meaningless.

